In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils.utilities import get_engine
import pandas as pd
import numpy as np
from scipy.stats import norm
import math

In [3]:
engine = get_engine()

In [4]:
stock_code = 'SSI'

In [5]:
query = """
    SELECT
        stock_code,
        date,
        close_price
    FROM public.transaction
    WHERE 
        stock_code = '{stock_code}'
"""
df = pd.read_sql_query(query, engine)
df

,stock_code,date,close_price


In [6]:
stocks = df['stock_code'].unique()
close_data = None
for stock in stocks:
    stock_data = df[df['stock_code'] == stock][['date', 'close_price']]
    stock_data.index = stock_data['date']
    stock_data = stock_data.drop('date', axis = 1).sort_index()
    stock_data = stock_data.rename(columns = {
        'close_price' : stock
    })
    if close_data is None:
        close_data = stock_data
    else:
        close_data = close_data.merge(stock_data, how = 'outer', left_index = True, right_index = True)

In [7]:
close_data.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
result = pd.DataFrame(columns = ['stock','sample_mean', 'interval_left', 'interval_right'])
for stock in close_data.columns:
    data = close_data[~close_data[stock].isnull()][stock]
    log_return = np.log(data.shift(-1)) - np.log(data) 
    sample_size = len(log_return)
    sample_mean = log_return.mean()
    sample_std = log_return.std(ddof = 1) / (sample_size ** 0.5)

    # left and right quantile
    z_left = norm.ppf(0.05)
    z_right = norm.ppf(0.95)

    # upper and lower bound
    interval_left = sample_mean + z_left * sample_std
    interval_right = sample_mean + z_right * sample_std

    # save result
    result.loc[len(result)] = [stock,pow(math.e, sample_mean), pow(math.e,interval_left), pow(math.e, interval_right)]


In [ ]:
result.sort_values(by = 'interval_left', ascending = False).to_csv('result.csv')